In [124]:
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import copy

In [197]:
def rot90_t(arr,n):
    for i in range(n):
        arr = np.rot90(arr)
    return(arr)
# create color map
cmap = plt.cm.get_cmap('RdYlGn', 4)  # red, yellow, green, white
cmap.set_under('w')  # set color for values less than 0 to white

colors = [(1,1,1),(0, 0, 0), (0.5, 0.5, 0.5), (1, 0, 0), (0, 0, 1), (1, 0.5, 0)]
# Define the levels for each color
levels = [ 0 ,1, 2, 3, 4, 5]
item_map = ListedColormap(colors, N=len(levels))

def place_box(box, place,i,j,show_plt = False):
    #global W_filp,B_filp,trans_times,i,j,major_b
    is_replace_box = np.clip(copy.deepcopy(place),0,1)
    combina_box = copy.deepcopy(place)

    is_replace_box[j:j+box.shape[0],i:i+box.shape[1]] = is_replace_box[j:j+box.shape[0],i:i+box.shape[1]] + np.clip(copy.deepcopy(box),0,1)
    mask = box >=1
    
    combina_box[j:j+box.shape[0],i:i+box.shape[1]][mask] = box[mask]

    if show_plt:
        fig, ax = plt.subplots(2,2)
        ax[0,0].imshow(place,cmap=item_map,vmin=0, vmax=5.9, interpolation='nearest')
        ax[0,1].imshow(combina_box,cmap=item_map,vmin=0, vmax=5.9, interpolation='nearest')
        ax[1,0].imshow(copy.deepcopy(place),cmap=cmap,vmin=0.1, vmax=4.9)
        ax[1,1].imshow(is_replace_box,cmap=cmap,vmin=0.1, vmax=4.9)
        
        #plt.show()
        plt.savefig('cover check\w{}_b{}_t{}_{}_{}_M{}.jpg'.format(W_filp,B_filp,trans_times,i,j,major_b))
        plt.close()
    return (is_replace_box,combina_box)

#1- production; 2- cheap production; 3- expensive; 4- tech; 5- promot
def record_box(replace_box,Combina_box,n):
    #global W_filp,B_filp,trans_times,i,j,major_b
    record_dict = dict()
    record_dict['n'] = n
    record_dict['replaced'] = (replace_box ==2).sum()
    record_dict['exposed'] = (replace_box ==1).sum()
    
    #result in gameplay #np.isin(arr, values)
    record_dict['production items'] = (Combina_box ==1).sum()
    record_dict['cheap production'] = (Combina_box ==2).sum()
    record_dict['price items'] = (Combina_box ==3).sum()
    record_dict['tech'] = (Combina_box == 4).sum()
    record_dict['promot'] = (Combina_box ==5).sum()
    record_dict['appear items'] = (Combina_box >0).sum()

    record_dict['base production'] = record_dict['production items'] +record_dict['cheap production'] +record_dict['replaced'] 
    record_dict['price'] = 5 + record_dict['price items'] - record_dict['cheap production']

    #placement information
    record_dict['W_side'] = W_filp
    record_dict['B_side'] = B_filp
    record_dict['B_trans_times'] = trans_times
    record_dict['B_placed_i'] = i
    record_dict['B_placed_j'] = j
    record_dict['major_box'] = major_b
    return (record_dict)
'''

box_B[0] = np.array([[2,2,2],[empty_label,2,2]]).transpose()
Combina_box = np.full((7, 7),0)
Combina_box[2:5,2:4]=1
place_box(box_B[0],Combina_box,1,2)'''


C:\Users\XSEA0081\AppData\Local\Temp\ipykernel_6580\4032262782.py:6: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap('RdYlGn', 4)  # red, yellow, green, white


'\n\nbox_B[0] = np.array([[2,2,2],[empty_label,2,2]]).transpose()\nCombina_box = np.full((7, 7),0)\nCombina_box[2:5,2:4]=1\nplace_box(box_B[0],Combina_box,1,2)'

In [307]:
empty_label = 0
#create 2 * 2 6 X 6 box
#1- production; 2- cheap production; 3- expensive; 4- tech; 5- promot
box_W = ['','']
box_B = ['','']
box_W[0] = np.array([[5,2,5],[empty_label,4,2]]).transpose()
box_W[1] = np.array([[empty_label,4,1],[4,3,5]]).transpose()
box_B[0] = np.array([[2,5,4],[empty_label,4,1]]).transpose()
box_B[1] = np.array([[empty_label,5,3],[3,5,4]]).transpose()

n = 1
Combina_box_record = dict()
result_df = pd.DataFrame()

#do matching combinations
#flip one, another one, and both
for W_filp in range(2):
    box_W_ues = box_W[W_filp]
    for B_filp in range(2):
#- centered one, and another transposed X3 one placement
        for trans_times in range(4):
            box_B_ues = rot90_t(box_B[B_filp],trans_times)
#- centered one, and another one placement
            for i in range(4):
                for j in range(5):
                    #print(n)
                    #print('w{}_b{}_t{}_{}_{}.jpg'.format(W_filp,B_filp,trans_times,i,j))

                    # create 9*8 empty boxes for combination
                    Combina_box = np.full((7, 7),empty_label)
                    replace_box, Combina_box = place_box(box_W_ues, Combina_box,2,2)
                    replace_box, Combina_box = place_box(box_B_ues, Combina_box,i,j)#,show_plt=True)
                    #print(Combina_box)

                    #invalid if covered count ==0 or 5
                    if  (replace_box ==2).sum() in [0,5]: 
                        continue
                    major_b = 'b'
                    record_dict = record_box(replace_box,Combina_box,n)
                    result_df = pd.concat([result_df,pd.DataFrame(record_dict,index=[n])])
                    Combina_box_record[n] = Combina_box
                    n=n+1
                    
#- all above but switch covering
                    null, Combina_box = place_box(box_W_ues, Combina_box,2,2)

                    major_b = 'w'
                    record_dict = record_box(replace_box,Combina_box,n)
                    result_df = pd.concat([result_df,pd.DataFrame(record_dict,index=[n])])
                    Combina_box_record[n] = Combina_box
                    n=n+1

# (3X5+4X4)*2*4*2 - not valid = (496 - not valid) combinations


#result in: 1 displayed items & covered count


In [199]:
for i in Combina_box_record:
    fig, ax = plt.subplots()
    ax.imshow(Combina_box_record[i],cmap=item_map,vmin=0, vmax=5.9, interpolation='nearest')
    plt.savefig('result in cover\{}.jpg'.format(i))
    plt.close()

In [220]:
result_df['check'] = result_df['exposed'] +result_df['replaced'] -result_df['appear items'] 
result_df['check'].max()

result_df[['W_side'	,'B_side',	'B_trans_times',	'B_placed_i',	'B_placed_j',	'major_box'	]].describe()
result_df.groupby(['W_side'	,'B_side',	'B_trans_times',	'B_placed_i',	'B_placed_j',	'major_box'	]).count().max()


n                   32
replaced            32
exposed             32
production items    32
cheap production    32
price items         32
tech                32
promot              32
appear items        32
base production     32
price               32
W_side              32
B_side              32
B_trans_times       32
major_box           32
check               32
dtype: int64

In [309]:
#base info
result_df.groupby('price')[['base production','tech','promot']].agg([max, min,'count','mean'])

base production                     tech                     promot      \
                  max min count      mean  max min count      mean    max min   
price                                                                           
2                   7   4    52  5.192308    3   1    52  2.288462      3   2   
3                   6   4    47  4.914894    3   2    47  2.659574      3   1   
4                   7   3    65  4.661538    4   1    65  2.138462      4   1   
5                   6   3    98  3.775510    4   1    98  2.683673      4   1   
6                   6   2    49  3.448980    4   1    49  2.469388      4   1   
7                   5   2    61  3.163934    3   1    61  2.360656      4   1   
8                   4   1    48  2.062500    3   1    48  2.395833      3   1   

                       
      count      mean  
price                  
2        52  2.519231  
3        47  2.425532  
4        65  2.584615  
5        98  2.214286  
6        49  2.510204  
7        61  2.475410  
8        48  2.541667

In [310]:

result_df[['price','base production','tech','promot']].sort_values(['price','base production','tech','promot'],ascending=False).to_csv('reviews.csv')

In [311]:
#any combination always better than other?
review_df = result_df[['price','base production','tech','promot']]
ddd = dict()
for case in review_df.index:
    compared_df = (review_df - review_df.loc[case])
    compared_df = compared_df[(compared_df['price']==0) & (compared_df['base production']>=0) &  (compared_df['tech']>=0) &  (compared_df['promot']>=0)]
    ddd[case]= (compared_df.max().sum() ==0)
review_df = pd.concat([review_df,pd.DataFrame.from_dict(ddd, orient='index',columns=['is optimal'])],axis=1)
review_df['unique'] = ~review_df.duplicated()
review_df

,price,base production,tech,promot,is optimal,unique
1,2,5,3,2,True,True
2,2,4,3,3,True,True
3,3,5,3,2,True,True
4,2,5,2,3,True,True
5,3,5,3,2,True,False
...,...,...,...,...,...,...
416,7,3,2,3,True,False
417,7,3,3,2,True,False
418,7,3,2,3,True,False
419,8,2,3,2,True,False


In [316]:
review_df.sum()

price              2100
base production    1635
tech               1026
promot             1026
is optimal          409
unique               58
dtype: int64

In [312]:
review_df[review_df['is optimal']*review_df['unique']].sort_values(['price','base production','tech','promot'],ascending=False)

,price,base production,tech,promot,is optimal,unique
408,8,4,2,1,True,True
358,8,3,3,1,True,True
320,8,3,2,2,True,True
375,8,3,1,3,True,True
331,8,2,3,2,True,True
317,8,2,2,3,True,True
323,8,1,3,3,True,True
352,7,5,2,1,True,True
330,7,4,3,1,True,True
328,7,4,2,2,True,True


In [286]:
review_df = result_df[['price','base production','tech','promot']]
compared_df = (review_df - review_df.loc[2])
compared_df = compared_df[(compared_df['price']==0) & (compared_df['base production']>=0) &  (compared_df['tech']>=0) &  (compared_df['promot']>=0)]
compared_df

,price,base production,tech,promot
2,0,0,0,0
32,0,0,0,0
76,0,0,0,0
92,0,0,0,0
100,0,0,0,0


In [265]:
review_df.loc[2,'is optimal'] = True
review_df

,price,base production,tech,promot,is optimal
1,2,5,3,2,True
2,2,8,3,3,True
3,3,5,3,2,NaN
4,2,8,2,3,NaN
5,3,5,3,2,NaN
...,...,...,...,...,...
416,7,6,2,3,NaN
417,7,3,3,2,NaN
418,7,6,2,3,NaN
419,8,2,3,2,NaN


In [244]:
cc[(cc['price']==0) & (cc['base production']>=0) &  (cc['tech']>=0) &  (cc['promot']>=0)]

,price,base production,tech,promot
1,0,0,0,0
2,0,3,0,1
7,0,0,0,0
9,0,0,0,0
15,0,0,0,0
25,0,0,0,0
31,0,0,0,0
32,0,3,0,1
33,0,0,0,0
34,0,4,0,0
